In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.float_format', lambda x: '%.1f' % x if not pd.isnull(x) else ' ')  # Display 1 decimal place, and empty for NaN
pd.set_option('expand_frame_repr', False)  # This line helps to display all columns

# define variables for past 5 years
ebit = np.array([60, 66, 72, 78, 85]) 
tax_rate = 0.3
capex = np.array([30, 33, 36, 40, 44])
depreciation = np.array([10, 11, 12, 13, 14])
roic = np.array([0.2, 0.21, 0.22, 0.23, 0.24])
risk_free_rate = 0.02 
equity_risk_premium = 0.05 
wacc = 0.08
revenue = np.array([100, 110, 120, 130, 140]) 


In [2]:
# calculate reinvestment rate and free cash flows for past 5 years
reinvestment_rate = (capex - depreciation) / (ebit * (1 - tax_rate))
ebit_margin = ebit / revenue

# Growth Rate
# Calculate initial growth rate for year 1-5
growth_rate_initial = reinvestment_rate[-1] * roic[-1]
# Create an array for the first 5 years with constant growth rate
growth_rates_1_to_5 = np.full(5, growth_rate_initial)
# Initialize the array for the next 5 years
growth_rates_6_to_10 = np.zeros(5)
# Set the growth rate for the 6th year
growth_rates_6_to_10[0] = growth_rate_initial - ((growth_rate_initial - risk_free_rate) / 5)
# Calculate growth rates for the 7th to 10th years
for i in range(1, 5):
    growth_rates_6_to_10[i] = growth_rates_6_to_10[i - 1] - ((growth_rates_6_to_10[i - 1] - risk_free_rate) / 5)
# Concatenate the two arrays
growth_rates = np.concatenate([growth_rates_1_to_5, growth_rates_6_to_10])

# growth rate applied at EBIT level
# Initialize array for future EBIT
ebit_future = np.zeros(10)
# Calculate EBIT for the first future year
ebit_future[0] = ebit[-1] * (1 + growth_rates[0])
# Calculate EBIT for the 2nd to 10th future years
for i in range(1, 10):
    ebit_future[i] = ebit_future[i - 1] * (1 + growth_rates[i])

revenue_future = ebit_future / ebit_margin[-1]

In [3]:
# create DataFrame to store all the data
columns = ['Year ' + str(i) for i in range(-4, 11)] + ['Terminal Value']
index = ['Revenue', 'Revenue Growth Rate (%)', 'EBIT', 'EBIT Margin (%)', 'EBIT (1 - tax rate)', 'Reinvestment', 'Free Cash Flows to Firm', 'Discount Rate (%)', 'Discount Factor', 'PV of Free Cash Flows']
df = pd.DataFrame(columns=columns, index=index)

In [4]:
# WACC 
# create two arrays: one for the first 5 years with constant WACC and one for the next 5 years with decreasing values
wacc_values_1_to_5 = np.full(5, wacc)
wacc_values_6_to_10 = np.linspace(wacc, risk_free_rate + equity_risk_premium, 5)
# concatenate the two arrays
wacc_values = np.concatenate([wacc_values_1_to_5, wacc_values_6_to_10])

# Historical data
for i in range(5):
    df.loc['Revenue', 'Year ' + str(i-4)] = revenue[i]
    df.loc['Revenue Growth Rate (%)', 'Year ' + str(i-4)] = (revenue[i] - revenue[i-1]) / revenue[i-1] * 100 if i != 0 else ""
    df.loc['EBIT', 'Year ' + str(i-4)] = ebit[i]
    df.loc['EBIT Margin (%)', 'Year ' + str(i-4)] = ebit_margin[i] * 100
    df.loc['EBIT (1 - tax rate)', 'Year ' + str(i-4)] = ebit[i] * (1 - tax_rate)
    df.loc['Reinvestment', 'Year ' + str(i-4)] = reinvestment_rate[i] * ebit[i]
    df.loc['Free Cash Flows to Firm', 'Year ' + str(i-4)] = ebit[i] * (1 - tax_rate) - reinvestment_rate[i] * ebit[i]

# calculate revenue, EBIT, and other values for the next 10 years
for i in range(10):
    if i == 0:
        df.loc['Revenue', 'Year ' + str(i+1)] = revenue_future[0]
        df.loc['Revenue Growth Rate (%)', 'Year ' + str(i+1)] = (revenue_future[0] - revenue[-1]) / revenue[-1] * 100
    else:
        df.loc['Revenue', 'Year ' + str(i+1)] = revenue_future[i]
        df.loc['Revenue Growth Rate (%)', 'Year ' + str(i+1)] = (revenue_future[i] - revenue_future[i-1]) / revenue_future[i-1] * 100

    df.loc['EBIT', 'Year ' + str(i+1)] = ebit_future[i]
    df.loc['EBIT Margin (%)', 'Year ' + str(i+1)] = ebit_margin[-1] * 100
    df.loc['EBIT (1 - tax rate)', 'Year ' + str(i+1)] = df.loc['EBIT', 'Year ' + str(i+1)] * (1 - tax_rate)
    df.loc['Reinvestment', 'Year ' + str(i+1)] = reinvestment_rate[-1] * df.loc['EBIT', 'Year ' + str(i+1)]
    df.loc['Free Cash Flows to Firm', 'Year ' + str(i+1)] = df.loc['EBIT (1 - tax rate)', 'Year ' + str(i+1)] - df.loc['Reinvestment', 'Year ' + str(i+1)]
    df.loc['Discount Rate (%)', 'Year ' + str(i+1)] = round(wacc_values[i] * 100, 2)
    df.loc['Discount Factor', 'Year ' + str(i+1)] = 1 / (1 + wacc_values[i]) ** (i+1)
    df.loc['PV of Free Cash Flows', 'Year ' + str(i+1)] = df.loc['Free Cash Flows to Firm', 'Year ' + str(i+1)] * df.loc['Discount Factor', 'Year ' + str(i+1)]

# calculate terminal values
terminal_growth_rate = risk_free_rate
terminal_reinvestment_rate = terminal_growth_rate / (risk_free_rate + equity_risk_premium)
terminal_free_cash_flows = df.loc['EBIT (1 - tax rate)', 'Year 10'] * (1 - terminal_reinvestment_rate)
terminal_value = terminal_free_cash_flows / (wacc_values[-1] - terminal_growth_rate)
df.loc['PV of Free Cash Flows', 'Terminal Value'] = terminal_value * df.loc['Discount Factor', 'Year 10']

In [5]:
# Total enterprise value is sum of all PV of cash flows
total_enterprise_value = df.loc['PV of Free Cash Flows'].sum()
df.loc['Enterprise Value'] = np.nan
df.loc['Enterprise Value', 'Year 0'] = total_enterprise_value
print('Total Enterprise Value: ', total_enterprise_value)

Total Enterprise Value:  1292.4205361063239


In [6]:
df = df.replace(np.nan, '', regex=True)

In [7]:
# Convert the DataFrame to an HTML table and use CSS to add borders and padding
df_html = df.to_html()
styled_df_html = f"""
<style>
.dataframe {{border: 1px solid black; border-collapse: collapse;}}
.dataframe th, .dataframe td {{text-align: right; padding: 10px; border: 1px solid black;}}
.dataframe th:nth-child(1), .dataframe td:nth-child(1) {{text-align: left;}}
.dataframe td:nth-child(n+2):nth-child(-n+6) {{color: grey;}}
.dataframe tr:last-child td {{color: black; font-weight: bold;}}
</style>
{df_html}
"""

# Display the HTML table
display(HTML(styled_df_html))

,Year -4,Year -3,Year -2,Year -1,Year 0,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6,Year 7,Year 8,Year 9,Year 10,Terminal Value
Revenue,100,110,120,130,140,156.9,175.9,197.2,221.1,247.8,272.8,295.9,317.1,336.6,354.5,
Revenue Growth Rate (%),,10.0,9.1,8.3,7.7,12.1,12.1,12.1,12.1,12.1,10.1,8.5,7.2,6.1,5.3,
EBIT,60,66,72,78,85,95.3,106.8,119.7,134.2,150.5,165.6,179.7,192.5,204.4,215.2,
EBIT Margin (%),60.0,60.0,60.0,60.0,60.7,60.7,60.7,60.7,60.7,60.7,60.7,60.7,60.7,60.7,60.7,
EBIT (1 - tax rate),42.0,46.2,50.4,54.6,59.5,66.7,74.8,83.8,94.0,105.3,116.0,125.8,134.8,143.1,150.7,
Reinvestment,28.6,31.4,34.3,38.6,42.9,48.0,53.9,60.4,67.7,75.9,83.5,90.6,97.1,103.0,108.5,
Free Cash Flows to Firm,13.4,14.8,16.1,16.0,16.6,18.7,20.9,23.4,26.3,29.5,32.4,35.2,37.7,40.0,42.1,
Discount Rate (%),,,,,,8.0,8.0,8.0,8.0,8.0,8.0,7.8,7.5,7.2,7.0,
Discount Factor,,,,,,0.9,0.9,0.8,0.7,0.7,0.6,0.6,0.6,0.5,0.5,
PV of Free Cash Flows,,,,,,17.3,17.9,18.6,19.3,20.1,20.4,20.9,21.1,21.3,21.4,1094.1
